In [31]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [25]:
from notebookutils import mssparkutils
from pyspark.sql.functions import lit, col, when
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType

In [4]:
año = name[0:4]
mes = name[4:6]

In [5]:
account_name = "alexstorage0003"
container_name = "datalake"
relative_path_in = f"silver/car_database/año={año}"
relative_path_out = "gold/car_database"

adls_path_in = 'abfss://%s@%s.dfs.core.windows.net/%s'%(container_name, account_name, relative_path_in)
adls_path_output = 'abfss://%s@%s.dfs.core.windows.net/%s'%(container_name, account_name, relative_path_out)

In [23]:
dataframe = spark.read.format("avro").load(adls_path_in)

In [28]:
dataframe = dataframe.select(col("placa"), \
                    when(col("marca").isin(['SUZUKI', 'MAZDA', 'RENAULT', 'HAVAL', 'GWM', 'GREAT WALL MOTORS', 'GREAT WALL MOTOR', 'CHANGAN', 'JAC']),\
                    "DERCO").otherwise(col("marca")).alias("marca"), \
                    col("modelo"), col("id_cliente").cast(IntegerType()), col("comuna"), col("region"), col("sexo"), col("actividad"), \
                    col("valor_vehiculo").cast(IntegerType()), col("fecha_transferencia").cast(IntegerType()),\
                    col("color"), col("edad"), col("vigencia"), col("mes"),  lit(f"{año}").cast(IntegerType()).alias("año"))

In [29]:
dataframe.printSchema()

In [32]:
dataframe.write\
    .mode('overwrite')\
    .format("parquet")\
    .partitionBy('año', 'mes')\
    .save(adls_path_output)